# Two Example of Calculating Euler Errors in a DSGE Model

### by [Kerk L. Phillips](https://sites.google.com/site/kerkphillips/home), July 2018

The code in this Jupyter notebook was written using Python 3.6.

This notebook explains how to caclualte the Euler errors for a simple DSGE model.  The model is solved and simulated two different ways: 1) using linearization about the steady state, amnd 2) using value-function iteration on a full cartesian grid.  This notebook uses a variety of downloadable code that can be found at the following Github site.  [https://github.com/kerkphil/DSGE-Utilities](https://github.com/kerkphil/DSGE-Utilities).

All the necessary files and this notebook can also be found in the public GitHub repository [https://github.com/kerkphil/Jupyter_Notebooks](https://github.com/kerkphil/Jupyter_Notebooks).

## The Model

Consider a model where an infinitely-lived representative agent solves the following dynamic program:
$$ V(k_t,z_t) = \max_{k_{t+1}} \frac{c_t^{1-\gamma} - 1}{1-\gamma} + \beta E \left\{ V(k_{t+1},z_{t+1}) \right\} $$
Subject to the following budget constraint:
$$ c_t = w_t + (1+r_t-\delta)k_t - k_{t+1} $$
The Euler equation from this optimization problem is:
$$ c_t^{-\gamma} = \beta E \left\{ c_{t+1}^{-\gamma} (1+r_{t+1}-\delta) \right\} $$

The representative firm solves the following static problem each period:
$$ \max_{K_t,L_t} \Pi = K_t^\alpha (e^{z_t} L_t)^{1-\alpha} - r_t K_t - w_t L_t $$
The solution to this problems yields the following:
$$ Y_t = K_t^\alpha (e^{z_t} L_t)^{1-\alpha} $$
$$ r_t = \alpha \frac{Y_t}{K_t} $$
$$ w_t = (1-\alpha) \frac{Y_t}{L_t} $$

The market clearing conditions are:
$$ k_t = K_t $$
$$ 1 = L_t $$

The law of motion for the technology deviations is:
$$ z_t = \rho z_{t-1} + \epsilon_t;\; \epsilon_t \sim iid(0,\sigma^2) $$

The state variables in this model are $k_t$ and $z_t$.

### The Definitions Function

We first define a definitions function that gives the non-state variables now as functions of the state variables now and last period.  Note that for notational purposes we have listed the current capital stock as $k_t$, even though this was chosen in the previous period.  $k_{t+1}$ is chosen in the current period.

In [1]:
def example_def(kp, k, z, param):
    # calculate definitions for GDP, wages, rental rates, consumption
    np.exp((1-alpha)*z)
    y = k**alpha*np.exp((1-alpha)*z)
    w = (1-alpha)*y
    r = alpha*y/k
    c = w + (1+r-delta)*k - kp
    i = y - c
    return y, c, i, r, w

### The Characterizing Equations

Next, we define an Euler equation written in the form:
$$ E \{ \Gamma(k_{t+2}, k_{t+1}, k_t, z_{t+1}, z_t) \} = 0 $$

In [2]:
def example_dyn(invec, param):
    # unpack invec
    kplus = invec[0]
    know = invec[1]
    kminus = invec[2]
    zplus = invec[3]
    znow = invec[4]

    # get definitions each period
    ynow, cnow, inow, rnow, wnow = example_def(know, kminus, znow, param)
    yplus, cplus, iplus, rplus, wplus = example_def(kplus, know, zplus, param)

    # calculate Gamma function
    Gamma = cnow**(-theta) / (beta*(cplus**(-theta)*(1+rplus-delta))) - 1.0
    
    return Gamma

In order to simulate and to calculate the Euler error we need a function for the law of motion for $z_t$

In [3]:
def example_lfunc(z, eps, lpars):
    zp = rho*z + sigma*eps
    return zp

Lastly, we define a function that takes the set $\{ k_{t+2}, k_{t+1}, k_t, z_{t+1}, z_t \} $ as individual items, rather than a vector, and returns the value of $\Gamma$ from above.  We will use this in calculating our Euler errors.

In [4]:
def example_efunc(kpp, kp, k, zp, z, epars):
    invec = np.array([kpp, kp, k[0], zp[0], z[0]])
    outvec = example_dyn(invec, epars)
    return outvec

## Euler Error Calculator

The Euler error is the devaition of the $\Gamma$ function from zero in a given period for the simulation.  Note that this function requires taking an expected value.

$$ E \{ \Gamma(k_{t+2}, k_{t+1}, k_t, z_{t+1}, z_t) \} = 0 $$

In order to calculate the Euler error a policy function for $k_{t+1}$.  The functional form will depend on the method we use to approximate it.  We write this function generally as:
$$ k_{t+1} = \Phi(k_t, z_t) $$

To calculate the Euler error we must use some form of quadrature to integrate over the possible values of $\epsilon_{t+1}$.  We denote the node values and their associated probabilites for this quadrature as $ \{\varepsilon_j, \phi_j\}_{j=1}^J $.

Since $k_t$, $z_t$ and $k_{t+1} are known today we need only substitute in the law of motion and the policy function to evaluate the Euler error

$$ e_t = \sum_{j=1}^J \phi_j \Gamma(\Phi(k_{t+1}, \rho z_t + \varepsilon_j), k_{t+1}, k_t, \rho z_t + \varepsilon_j, z_t) \} $$

We now load Python code that calculates Euler errors.  This code is available at [https://github.com/kerkphil/DSGE-Utilities](https://github.com/kerkphil/DSGE-Utilities).  However, we recreate it here to illustrate how the Euler errors are calculated along the lines discussed above.

In [5]:
import numpy as np
from scipy.stats import norm

def EEcalc(Xdata, Zdata, efunc, epars, tfunc, tpars, lfunc, lpars):
    '''
    This function calculates Euler Errors for a DSGE model.
    It takes the following as inputs:
    1)  Xdata: an nobs-by-nx matrix of endogenous state variable data with with 
        nobs indexing the observations in rows and nx indexing the variables in 
        columns.
    2)  Zdata: an nobs-by-nz matrix of exogenous state variable data with with 
        nobs indexing the observations in rows and nz indexing the variables in 
        columns.
    3)  efunc: the name of a function which takes a single observation of data 
        and returns the Euler error for a given realization of future exogenous 
        variables.  This function must take 3 nx-element vectors of endogenous
        state variables and and 2 nz-element vector of exogenous state 
        variables as inputs and output an neq-element vector of Euler equation 
        errors.  The order of input is X(t+2), X(t+1), X(t), Z(t+1), Z(t),
        epars
    4)  epars: a list of parameters passed to efunc.
    5)  tfunc: the name of a transition function which takes a single 
        observation of the state variables and returns next period's value of 
        the endogenous state variables.  This function must take an nx-element 
        vector of endogenous state variables and and nz-element vector of 
        exogenous state variables as inputs and output an nz-element vector of 
        next-period endogenous state variables.  The order of inputs is X(t), 
        Z(t), tpars
    6)  tpars: a list of parameters passed to tfunc.
    7)  lfunc: the name of a law-of-motion function which takes a single 
        observation of the exogenous state variables and returns next period's 
        value of the exogenous state variables.  This function must take an 
        nz-element of exogenous state and an scalar iid shock as inputs 
        and output an nz-element vector of next-period endogenous state 
        variables. The order of inputs is Z(t), Eps(t+1), epars
    8)  lpars: a list of parameters passed to lfunc.
    
    The function returns the following outputs:
    1)  Eerr: an nobs-by-neq matrix of Euler errors with nobs indexing the 
        observations in rows and neq indexing the elements from the function 
        efunc in columns.
        
    Notes:
    Xdata and Zdata must have the same number of rows.
    Neither Xdata nor Zdata may have missing, nan, or complex values. 
    Innovations to the law of motion are drawn from a standard normal 
    distribution.
    Currently this function only works with one innovation shock, i.e. ne=1

    To Do:
    1) Allow for more than one shock process.  May require the use of sparse 
    grids for quadrature.
    2) Use a more efficient quadrature method.  Gaussian?
    '''
    
    # set parameter values
    npts = 10 # number of point for rectangular quadrature
    
    # check sizes of data matrices
    (Xnobs, nx) = Xdata.shape
    (Znobs, nz) = Zdata.shape
    if Xnobs == Znobs:
        nobs = Xnobs
    else:
        print ('Data matrices have different numbers of observations')
        nobs = min(Xnobs, Znobs)
        Xdata = Xdata[0:nobs]
        Zdata = Zdata[0:nobs]
    
    # generate discret support for epsilon to be used in Euler error
    # Eps are the central values
    # Phi are the associated probabilities
    Eps = np.zeros(npts);
    Cum = np.linspace(0.0, 1.0, num=npts+1)+.5/npts
    Cum = Cum[0:npts]
    Phi = np.ones(npts)/npts
    Eps = norm.ppf(Cum)

    neq = nx
    
    # initialize matrix of Euler errors
    Eerr = np.zeros((nobs,neq))
    # begin loop over time periods
    for t in range(0, nobs):
        # begin loop over possible va,lues of shock next period
        for i in range(0, npts):
            # find value of next period Z
            Zp = lfunc(Zdata[t,:],Eps[i],lpars)
            # find the value of X next period
            Xp = tfunc(Xdata[t,:],Zdata[t,:],tpars)
            # find the value of X in two periods
            Xpp = tfunc(Xp,Zp,tpars)
            # find the Euler errors
            Eerr[t,:] = Eerr[t,:] + Phi[i]*efunc(Xpp,Xp,Xdata[t,:], \
                Zp,Zdata[t,:],epars)
    return Eerr

## Solving and Simulating Using Linearization

To linearize we use the LinApp package available at [https://github.com/kerkphil/DSGE-Utilities](https://github.com/kerkphil/DSGE-Utilities).

The LinApp package includes the following files:

LinApp_FindSS - Takes a function, funcname.py, and a steady state value for the exogenous state variables as inputs.  Finds the steady state for the model's endogenous state and jump variables.

LinApp_Deriv - Takes a function, funcname.py, as an input.   Generates the derivative matrices needed for the Uhlig toolkit.

LinApp_Solve - Takes the derivative matrices as inputs.  Uses the code from the Uhlig toolkit to generate the coefficients for the linearized policy function(s).

LinApp_Sim - Takes the policy function coefficients and current state as inputs.  Generates the value(s) of the endogenous state variables for the next period.

LinApp_SSL - Takes a history of exogenous state variables and the policy function coefficients as inputs.  Generates a history of state and jump variables using linearization about the steady state.

We first define our policy or transition function for the linearized case.  This is:

$$ k_{t+1} = \bar k + P (k_t - \bar k) + Q z_t $$

The LinApp package will find the values of $\bar k$, $P$ and $Q$.

In [6]:
 def example_tfunc(k, z, tpars):
    kp = PP*(k-kbar) + QQ*z + kbar
    return kp

We now proceed with our main program

In [7]:
from LinApp_FindSS import LinApp_FindSS
from LinApp_Deriv import LinApp_Deriv
from LinApp_Solve import LinApp_Solve

# set parameter values
#  model parameters
delta = .08
alpha = .33
theta = 2.5
beta = .9
rho = .9
sigma = .0075
params = [delta, alpha, theta, rho, sigma, beta]

#  other parameters
nobs = 300       # number of periods in simulatipon
kstart = 1       # starting value for simulation (proportional to kbar)
nx = 1           # number of endogenous state variables
ny = 0           # number of jump variables
nz = 1           # number of exogenous state variables
takelogs = False # if True log-linearize, otherwise simple linearization
useSylv = False  # if ture use a built-in Sylvester equation solver

# find the steady state
guessk= np.array([1.])
zbar = np.array([0.])
kbar = LinApp_FindSS(example_dyn, params, guessk, zbar, nx, ny)
# convert from numpy array to real
kbar = kbar[0]
# check that the solution works
invec = np.array([kbar, kbar, kbar, 0., 0.])
check = example_dyn(invec, params)
print('kbar is ', kbar)
print('check should be close to zero and is ', check)

# find derivatives
AA, BB, CC, DD, FF, GG, HH, JJ, KK, LL, MM = \
    LinApp_Deriv(example_dyn, params, invec, nx, ny, nz, takelogs)

# find policy function coefficients
PP, QQ, RR, SS = \
    LinApp_Solve(AA, BB, CC, DD, FF, GG, HH, JJ, KK, LL, MM, \
                 rho, zbar, useSylv)

print('P is ', PP)
print('Q is ', QQ)

kbar is  2.259808539186245
check should be close to zero and is  2.220446049250313e-16
P is  [[0.90121425]]
Q is  [[0.40395128]]


We now simulate our model

In [8]:
# perform simulation
tpars = (PP, QQ, kbar)
lpars = (rho)
eps = np.random.randn(nobs)*sigma
z = np.zeros((nobs+1))
k = np.zeros((nobs+1))
y = np.zeros(nobs)
r = np.zeros(nobs)
w = np.zeros(nobs)
i = np.zeros(nobs)
c = np.zeros(nobs)
k[0] = kbar*kstart
z[0] = eps[0]
for t in range(0, nobs):
    z[t+1] = example_lfunc(z[t], eps[t], lpars)
    k[t+1] = example_tfunc(k[t], z[t], tpars)
    y[t], c[t], i[t], r[t], w[t] = example_def(k[t+1], k[t], z[t], params)
k = k[0:nobs]
z = z[0:nobs]

Finally, we calculate our Euler errors using the `EEcalc` function from above.

In [9]:
# find Euler Errors
Xdata = k.reshape(len(k), 1)
Zdata = z.reshape(len(z), 1)
efunc = example_efunc
epars = params
tfunc = example_tfunc
lfunc = example_lfunc
EErrs = EEcalc(Xdata, Zdata, efunc, epars, tfunc, tpars, lfunc, lpars)

MaxAbsEE = np.max(np.abs(EErrs))
MeanAbsEE = np.mean(np.abs(EErrs))
RootMeanSqEE = (np.mean(EErrs**2))**.5

print('Euler Error Summary Statistics')
print('Maximum Absolute Euler Error:', MaxAbsEE)
print('Mean Absolute Euler Error:', MeanAbsEE)
print('Root Mean Squared Euler Error:', RootMeanSqEE)

Euler Error Summary Statistics
Maximum Absolute Euler Error: 4.132128849467756e-05
Mean Absolute Euler Error: 4.124359048024828e-05
Root Mean Squared Euler Error: 4.124421047312285e-05


## Solving and Simulating Using Value-Function Iteration

The policy or transtion function in this case could be to take values straight from the discrete verision of the policy function calculated over the grid.  Or we could use a continuous polynomial function fitted to that.  This is shown below with the option `gridsim` indicating which is used.

In [10]:
def example_tfunc2(k, z, tpars):
    k = np.reshape(k,(1))
    z = np.reshape(z,(1))
    if gridsim:
        # perform simulation with grid
        kidx = np.argmin(np.abs(kvec-k)) # index of closest value for k
        zidx = np.argmin(np.abs(zvec-z)) # index of closest value for z
        kp = trans[zidx,kidx];
    else:
        # perform simulation with polynomial fit
        temp =  np.stack(([1.0], k, k**2, k**3, z, z**2, z**3, k*z, k**2*z, \
                              k*z**2))
        kp = np.dot(coeffs,temp)
    return kp

We use many of the same model parameters as before, but must define new ones specific to value-function iteration

In [11]:
low = .2           # low end of k grid (proportional to kbar)
high = 5           # high end of k grid (proportional to kbar)
spread = 3         # spread of the z grid (proportional to sigma)
nptsk = 21         # number of points in the k grid
nptsz = 21         # number of points in the z grid
converge = .0001;  # convergence criterion for value-function iteration
gridsim = 0        # set to 1 to use grid, 0 to use polynomial fit

We now find the policy function

In [12]:
# set up grid for k
klow = low*kbar   # low end of grid
khigh = high*kbar # high end of grid
kincr = np.log(khigh/klow)/(nptsk-1);
kvec = np.zeros(nptsk)
kvec[0] = klow
for i in range (1, nptsk):
    kvec[i] = kvec[i-1]*(1+kincr)

# set up grid for z
zlow = -spread*max(sigma, .001)
zhigh = spread*max(sigma, .001)
zincr = (zhigh-zlow)/(nptsz-1);
zvec = np.zeros(nptsz)
zvec[0] = zlow
for i in range (1, nptsz):
    zvec[i] = zvec[i-1] + zincr

# create meshgrid
kmesh, zmesh = np.meshgrid(kvec, zvec)

# find value function and transition function
distance = 1.0;
maxwhile = 1000;
count = 0;
value = np.zeros((nptsz,nptsk));
newval = np.zeros((nptsz,nptsk));
trans = np.zeros((nptsz,nptsk));
while distance > converge:
    count = count + 1
    if count > maxwhile:
        break
    for i in range(0, nptsz): # for all values of z(t)
        # find index of closest value for E{z(t+1)}
        m = np.argmin(np.abs(zvec - zvec[i]*rho))
        for j in range(0, nptsk): # for all values of k(t)
            maxval = -10000000000;
            for l in range(0, nptsk): # search over all values of k(t+1)
                if theta == 1:
                    temp = np.log(kvec[j]**alpha*np.exp((1-alpha)*zvec[i]) 
                    +(1-delta)*kvec[j]-kvec[l]) + beta*value[m,l]
                else:
                    temp = (((kvec[j]**alpha*np.exp((1-alpha)*zvec[i]) 
                    +(1-delta)*kvec[j]-kvec[l])**(1-theta)-1)) \
                    / (1-theta) + beta*value[m,l]
                # print i, j, temp
                if np.iscomplex(temp):
                   temp = -100000000
                if np.isnan(temp):
                    temp = -100000000
                if temp > maxval:
                    maxval = temp
                    newval[i,j] = temp
                    trans[i,j] = kvec[l]
    # print newval
    distance = np.mean(np.abs(value/newval - 1.0))
    print('count = ', count, 'distance = ', distance)
    for i in range(0, nptsz):
        for j in range(0, nptsk):
            value[i,j] = newval[i,j]

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in double_scalars


count =  1 distance =  1.0
count =  2 distance =  0.5452517012577935
count =  3 distance =  0.3558578751497489
count =  4 distance =  0.6402419193387543
count =  5 distance =  0.18632670588729233
count =  6 distance =  0.457206920739312
count =  7 distance =  0.27937124710057004
count =  8 distance =  0.11249655568784568
count =  9 distance =  0.06774281340580439
count =  10 distance =  0.05504814003122724
count =  11 distance =  0.054319684882198786
count =  12 distance =  0.1882708196999667
count =  13 distance =  0.06565295655067123
count =  14 distance =  0.06296436620907432
count =  15 distance =  0.04568130780380501
count =  16 distance =  0.5817832636324972
count =  17 distance =  0.02512202815731057
count =  18 distance =  0.021323751656452176
count =  19 distance =  0.020008745300104862
count =  20 distance =  0.016689417863437788
count =  21 distance =  0.01384304130664234
count =  22 distance =  0.04043552605459304
count =  23 distance =  0.01179425830371814
count =  24 dist

Fit the policy function with a polynomial

In [13]:
 # fit a polynomial
Y = trans.flatten()

X = np.ones(nptsk*nptsz)

temp = kmesh.flatten()
X = np.vstack((X,temp))

temp = kmesh**2
temp = temp.flatten()
X = np.vstack((X,temp))

temp = kmesh**3
temp = temp.flatten()
X = np.vstack((X,temp))

temp = zmesh.flatten()
X = np.vstack((X,temp))

temp = zmesh**2
temp = temp.flatten()
X = np.vstack((X,temp))

temp = zmesh**3
temp = temp.flatten()
X = np.vstack((X,temp))

temp = kmesh*zmesh
temp = temp.flatten()
X = np.vstack((X,temp))

temp = kmesh**2*zmesh
temp = temp.flatten()
X = np.vstack((X,temp))

temp = kmesh*zmesh**2
temp = temp.flatten()
X = np.vstack((X,temp))

XtX = np.dot(X,np.transpose(X))
XtY = np.dot(X,Y)
coeffs = np.dot(np.linalg.inv(XtX),XtY)
tpoly = np.zeros((nptsz,nptsk))
for i in range(0, nptsz):
    for j in range(0, nptsk):
        tpoly[i,j] = np.dot(np.stack((1, kvec[j], kvec[j]**2, kvec[j]**3, \
        zvec[i], zvec[i]**2, zvec[i]**3, \
        kvec[j]*zvec[i], kvec[j]**2*zvec[i], kvec[j]*zvec[i]**2)),coeffs)

# calcuate R-squared
Rsq = 1 - np.sum((trans-tpoly)**2)/np.sum(trans**2)
print('R-squared', Rsq)

R-squared 0.999335409097344


We now simulate our model

In [14]:
# perform simulation
eps = np.random.randn(nobs)*sigma
z = np.zeros(nobs+1)
k = np.zeros(nobs+1)
k[0] = kstart*kbar
z[0] = eps[0]
tpars = [kvec, zvec, trans, coeffs]
lpars = [rho, sigma]
for t in range(0, nobs):
    k[t+1] = example_tfunc(k[t], z[t], tpars)
    z[t+1] = example_lfunc(z[t], eps[t], lpars)
    
# remove final k & z
k = k[0:nobs]
z = z[0:nobs]

Finally, we again calculate our Euler errors using the `EEcalc` function from above.

In [15]:
# find Euler Errors
Xdata = k.reshape(len(k), 1)
Zdata = z.reshape(len(z), 1)
efunc = example_efunc
epars = params
tfunc = example_tfunc2
lfunc = example_lfunc
EErrs = EEcalc(Xdata, Zdata, efunc, epars, tfunc, tpars, lfunc, lpars)

MaxAbsEE = np.max(np.abs(EErrs))
MeanAbsEE = np.mean(np.abs(EErrs))
RootMeanSqEE = (np.mean(EErrs**2))**.5

print('Euler Error Summary Statistics')
print('Maximum Absolute Euler Error:', MaxAbsEE)
print('Mean Absolute Euler Error:', MeanAbsEE)
print('Root Mean Squared Euler Error:', RootMeanSqEE)

Euler Error Summary Statistics
Maximum Absolute Euler Error: 0.019506309755123973
Mean Absolute Euler Error: 0.01899616624415117
Root Mean Squared Euler Error: 0.01899640982371943
